In [55]:
!pip install transformers
!pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [56]:
from google.colab import drive
from torch import nn
from transformers import BertTokenizer, BertModel
import torch
import csv
from torch.utils.data import Dataset
from transformers import BertTokenizer, BertModel
from torchmetrics import Accuracy
import os
from torch.utils.data.dataloader import DataLoader
from torch.nn.utils.rnn import pad_sequence
import tqdm

In [57]:
drive.mount('/content/drive')

data_path = '/content/drive/MyDrive/data'

train_data_path = f'{data_path}/train'
true_train_path = f'{train_data_path}/true.csv'
false_train_path = f'{train_data_path}/false.csv'

test_data_path = f'{data_path}/test'
true_test_path = f'{test_data_path}/true.csv'
false_test_path = f'{test_data_path}/false.csv'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [58]:


class TextClassificationModel(nn.Module):

    def __init__(self, embed_dim=768, num_class=1):
        super(TextClassificationModel, self).__init__()
        self.bert = BertModel.from_pretrained("bert-base-uncased").cuda()
        self.fc = nn.Linear(embed_dim, num_class).cuda()
        self.sigmoid = nn.Sigmoid().cuda()

    def forward(self, input_ids, attention_masks):
        inputs = {
            "input_ids": input_ids.cuda(), 
            "attention_mask" : attention_masks.cuda()
        }
        outputs = self.bert(**inputs)
        output = outputs.last_hidden_state
        output = output[:,0,:]
        output = self.fc(output)
        return self.sigmoid(output)



In [59]:
class SpellCheckingDataset(Dataset):

    def __init__(self, tokenizer, data_paths, labels, batch_size=32):
        self.dataset = []
        for i in range(len(data_paths)):
          data_path = data_paths[i]
          with open(data_path, 'r', encoding='utf-8') as file:
              data = csv.reader(file)
              for item in data:
                self.dataset.append((item[0], labels[i]))
        self.tokenizer = tokenizer
        self.batch_size = batch_size

    def __getitem__(self, idx):
        if (self.tokenizer == None):
          raise Exception('Tokenizer cannot be null')

        tweet, label = self.dataset[idx]
        tokenized_tweet = self.tokenizer(tweet[0])
        input_ids = tokenized_tweet['input_ids']
        attention_mask = tokenized_tweet['attention_mask']

        return input_ids, attention_mask, label

    def __len__(self):
      return len(self.dataset)

In [60]:
CHECKPOINTS_DIR = '/content/models'

def pad_batched_sequence(batch):
    
    input_ids = [torch.tensor(item[0]) for item in batch]
    # print('input ids')
    # print(input_ids)

    attention_masks =  [torch.tensor(item[1]) for item in batch]
    # print('attention masks')
    # print(attention_masks)

    input_ids = pad_sequence(input_ids, padding_value=0, batch_first=True)
    # print('pad input ids')
    # print(input_ids)

    attention_masks = pad_sequence(attention_masks, 
                                   padding_value=0, 
                                   batch_first=True)
    # print('pad attention masks')
    # print(attention_masks)

    labels = None
    
    if batch[0][2]:
        labels = torch.tensor([[item[2]] for item in batch]).double().cuda()
    
    return input_ids.cuda(), attention_masks.cuda(), labels

class SpellCheckingTrainer():

  def __init__(self,
               model,
               train_dataset,
               save_data_path,
               batch_size=32,
               epochs=20,
               lr=0.001):
        
    self.model = model
    self.epochs = epochs
    self.batch_size = batch_size
    self.train_loader = DataLoader(train_dataset, 
                                   batch_size=batch_size, 
                                   shuffle=True, 
                                   drop_last=True, 
                                   collate_fn=pad_batched_sequence)
    self.save_path = save_data_path
    self.loss_function = nn.BCELoss()
    self.optimizer = torch.optim.Adam(list(self.model.parameters()), lr=lr)        
    self.accuracy = Accuracy(num_classes=1)

  def train_one_epoch(self, epoch_index):
    running_loss = 0.
    running_accuracy = 0.
    last_loss = 0.
    threshold = torch.tensor([0.5]).cuda()

    for i, data in tqdm.tqdm(enumerate(self.train_loader), 
                             total=len(self.train_loader)):
        # Every data instance is an input + label pair
        input_ids, attention_masks, labels = data

        # Zero your gradients for every batch!
        self.optimizer.zero_grad()

        # Make predictions for this batch
        outputs = self.model(input_ids, attention_masks)

        # Compute the loss and its gradients
        loss = self.loss_function(outputs.double(), labels.double())
        loss.backward()

        # Adjust learning weights
        self.optimizer.step()

        # Gather data and report
        running_loss += loss.item()
        
        result = (outputs > threshold).float() * 1
        running_accuracy += torch.sum(result == labels) / self.batch_size
        
        if i % 10 == 9:
            last_loss = running_loss / 10 # loss per batch
            last_accuracy = running_accuracy / 10 # accuracy per batch
            print('  batch {} loss: {}'.format(i + 1, last_loss))
            print('  batch {} accuracy: {}'.format(i + 1, last_accuracy))
            running_loss = 0.
            running_accuracy = 0.

    return last_loss


  def train(self):
    epoch_number = 0
    for epoch in range(self.epochs):
        print('EPOCH {}:'.format(epoch_number + 1))

        # Make sure gradient tracking is on, and do a pass over the data
        self.model.train(True)
        avg_loss = self.train_one_epoch(epoch_number)

        # We don't need gradients on to do reporting
        self.model.train(False)

        epoch_number += 1
    
    torch.save(self.model.state_dict(), self.save_path)
    
          
          

In [ ]:


tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")

true_label = 1
false_label = 0

true_train_dataset = SpellCheckingDataset(tokenizer, [true_train_path], [true_label])
false_train_dataset = SpellCheckingDataset(tokenizer, [false_train_path], [false_label])

test_dataset = SpellCheckingDataset(tokenizer, 
                                    [true_test_path, false_test_path],
                                    [true_label, false_label])

true_saved_model_path = '/content/drive/MyDrive/models/true_bert.berm_lm'
false_saved_model_path = '/content/drive/MyDrive/models/false_bert.berm_lm'

true_trainer = SpellCheckingTrainer(TextClassificationModel(), 
                                    true_train_dataset,
                                    true_saved_model_path)

false_trainer = SpellCheckingTrainer(TextClassificationModel(), 
                                     false_train_dataset,
                                     false_saved_model_path)
 

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predic

In [ ]:
true_trainer.train()
false_trainer.train()

# model = TextClassificationModel()
# model.load_state_dict(torch.load(true_saved_model_path))
# model.eval()